In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [3]:
X_full = pd.read_csv('train.csv', index_col='id')
X_test_full = pd.read_csv('test.csv', index_col='id')

In [4]:
X_full.dropna(subset=['target'], axis=0, inplace=True)
y = X_full.target
X_full.drop('target', axis=1, inplace=True)

In [5]:
from sklearn.model_selection import train_test_split

In [13]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, test_size=0.2, random_state=1)

In [14]:
num_cols = [col for col in X_train_full.columns if X_train_full[col].dtypes in ['int64', 'float64']]

cat_cols = [col for col in X_train_full.columns if X_train_full[col].nunique() < 10 and X_train_full[col].dtype == 'object']

In [15]:
my_cols = num_cols + cat_cols

X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# Train

In [16]:
num_trans = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),
                            ('std_scaler', StandardScaler())])

cat_trans = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                            ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [17]:
preprocessor = ColumnTransformer(transformers=[('num', num_trans, num_cols),
                                               ('cat', cat_trans, cat_cols)])

In [18]:
my_model = RandomForestRegressor(n_estimators=100, random_state=1)

In [19]:
models = Pipeline(steps=[('preprocessor', preprocessor), ('model', my_model)])

In [20]:
models.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('std_scaler',
                                                                   StandardScaler())]),
                                                  ['cont0', 'cont1', 'cont2',
                                                   'cont3', 'cont4', 'cont5',
                                                   'cont6', 'cont7', 'cont8',
                                                   'cont9', 'cont10', 'cont11',
                                                   'cont12', 'cont13']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                        

In [21]:
pr = models.predict(X_train)
preds = models.predict(X_valid)
print('RMSE:', mean_squared_error(y_train, pr, squared=False))
print('RMSE:', mean_squared_error(y_valid, preds, squared=False))

RMSE: 0.27532106087588787
RMSE: 0.7330639419445089


In [ ]:
RMSE: 0.2753472447284073
RMSE: 0.7331196969184158

In [22]:
preds_test = models.predict(X_test)

In [23]:
output = pd.DataFrame({'id': X_test.index, 'target': preds_test})

output.to_csv('submission.csv', index=False)

In [24]:
X_train.head()

,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,...,cont13,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8
id,,,,,,,,,,,,,,,,,,,,,
298173,0.558652,0.364985,0.953177,0.261481,0.285042,0.366598,0.552537,0.242170,0.427087,0.413445,...,0.276961,A,A,A,C,B,B,A,E,C
299785,0.542071,0.309224,0.736795,0.455526,0.475618,0.386943,0.334639,0.256481,0.441711,0.423982,...,0.161990,A,B,B,C,B,B,A,E,C
141896,0.398638,0.619891,1.003411,0.543077,0.458128,0.361442,-0.179812,0.649192,0.408889,0.282222,...,0.256212,A,B,A,C,B,D,A,E,C
214015,0.079594,0.216732,0.343160,0.426862,0.274832,0.648311,0.568285,0.768072,0.569046,0.781905,...,0.170625,A,B,A,C,B,D,A,E,A
82082,0.514264,0.495396,0.602715,0.455140,0.964888,0.495950,0.432111,0.380077,0.264304,0.280083,...,0.289577,A,B,A,C,B,B,A,E,E
